# Google Earth Engine: JavaScript, but Python

[Google Earth Engine (GEE)](https://earthengine.google.com/) is an online tool that combines a huge catalogue of satellite imagery and other geospatial datasets with Google's processing capabilities to reduce processes that before would take days to only seconds.  

GEE API (Application Programming Interface) is available in JavaScript (the original one) and Python.





The GEE JavaScript API is available through the [Code Editor](https://code.earthengine.google.com/#workspace), and while it is very useful to get started with GEE, it is limited in terms of processing power (time limitations, slower).


The GEE Python API can be loaded via a library (`ee`), and combines the power of Python with everything else you can do via the JavaScript API. 





The only "issue" with the Python API: ALL of the GEE [documentation](https://developers.google.com/earth-engine/) and [tutorials](https://developers.google.com/earth-engine/tutorials) are developed in JavaScript!!




Which means there are two possible routes:


1.   The painful one (the one I took): learn everything in JavaScript and THEN transfer to Python (NOT RECOMMENDED!).
2.   The sensible one, but slower: to learn the Pythonic way from the start.

We are only going to use Python in this course, but it is very useful to have all the documentation,e ven if it is JavaScript, as we can see how things are done and translate them!

Note: we are going to jump directly to Python, but here we show how some things can be expressed in JavaScript and their equivalent in Python. And remember, if you want to use the [GEE online code editor](https://code.earthengine.google.com/), it has to be JavaScript style! 

I recommend working from Google Colab (using Google's cloud computing services for free!!) and the GEE Python API from the start.

We are using the GEE Guides, but online you will find them in JavaScript, while here we have the Python version. You should go through the GEE guides and tutorials in your own time, and try to transfer to Python for consistency.

## `Maps` warning!

The GEE JavaScript does not allow loops or conditionals. Instead, you will have to use the `Map` function. This is a huge limitation , particularly when we want to work with satellite data and other big datasets, machine learning applications, etc.

Keep that in mind if you use the GEE Code Editor.

# Some basic GEE concepts

Import the GEE API:

In [1]:
import ee

Authenticate and initialise:

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Image constructor: ee.Image

Raster data is represented as Image objects in Earth Engine. 

Images are composed of one or more bands and each band has its own name, data type, scale, mask and projection. Each image has metadata stored as a set of properties.

Images can be loaded by pasting an Earth Engine asset ID into the ee.Image constructor. You can find image IDs in the data catalog. For example, to load JAXA's ALOS DSM:

In [3]:
loadedImage = ee.Image('JAXA/ALOS/AW3D30/V2_2')

/exports/csce/datastore/geos/users/s2112771/miniforge3/envs/sensebook/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for JAXA/ALOS/AW3D30/V2_2! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_AW3D30_V2_2

  warnings.warn(warning, category=DeprecationWarning)


Note that finding an image through the GEE Code Editor search tool is equivalent. When you import the asset, the image construction code is written for you in the imports section of the Code Editor. You can also use the asset ID (bottom left when you search online).

## Visualising images and image bands

## Get an ee.Image from an ee.ImageCollection

The standard way to get an image out of a collection is to load the collection, filter the collection, with filters in order of decreasing specificity. For example, to get an image out of the Sentinel-2 surface reflectance collection (level 2A):

In [4]:
# Image near Acra (Ghana)
first = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(ee.Geometry.Point(0, 6)).filterDate('2019-01-01', '2019-12-31').sort('CLOUDY_PIXEL_PERCENTAGE').first()      

/exports/csce/datastore/geos/users/s2112771/miniforge3/envs/sensebook/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


## Display static image

In [5]:
from IPython.display import Image

Image(url=first.getThumbUrl({'min': 0, 'max': 2000, 'bands':['B4','B3','B2'],'dimensions': 512}))

## Constant images

In addition to loading images by ID, you can also create images from constants, lists or other suitable Earth Engine objects. The following illustrates methods for creating images, getting band subsets, and manipulating bands:

In [6]:
# Create a constant image
image1 = ee.Image(1)
print("image1:",image1.getInfo())

# Concatenate two images into one multi-band image.
image2 = ee.Image(2)
image3 = ee.Image.cat([image1, image2])
print("image3:",image3.getInfo())

# Create a multi-band image from a list of constants.
multiband = ee.Image([1, 2, 3]);
print("mutiband:",multiband.getInfo())

# Select and (optionally) rename bands.
renamed = multiband.select(['constant', 'constant_1', 'constant_2'],['band1', 'band2', 'band3'])
print("renamed:",renamed.getInfo())

# Add bands to an image.
image4 = image3.addBands(ee.Image(42))
print("image4:",image4.getInfo())

image1: {'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
image3: {'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
mutiband: {'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 3, 'max': 3}, 'crs': 'EPSG:4326', 'c

## RGB composites

The following illustrates the use of parameters to style a Landsat 8 image as a false-color composite:

In [8]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# ANOTHER WAY TO DO IT
#Define the visualization parameters.
#vizParams = {'min': 0,  'max': 0.5,  'gamma': [0.95, 1.1, 1]}
#import ee.mapclient
#ee.mapclient.centerMap(-122.1899, 37.5010, 9) # San Francisco Bay
#ee.mapclient.addToMap(image.select('B5','B4','B3'), vizParams, 'False colour composite')

# Display the image normally.
Image(url=image.getThumbUrl({'bands':['B5','B4','B3'],'min': 0,  'max': 0.5,'dimensions': 512}))

In this example, band ‘B5’ (NIR) is assigned to red, ‘B4’ (Red) is assigned to green, and ‘B3’ (Green) is assigned to blue, creating the false colour composite.

False color composites allow us to visualize the wavelengths the human eye does not see (near the infrared range). The use of bands, such as near infrared, increases spectral separation and can enhance the interpretability of data. 

## Colour palettes

To display a single band of an image in colour, set the palette parameter with a colour ramp represented by a list of CSS-style color strings.

The following example illustrates how to use colors from cyan (‘00FFFF’) to blue (‘0000FF’) to render a Normalized Difference Water Index (NDWI) image:

In [9]:
# Create an NDWI image from previous image, define visualization parameters and display
ndwi = image.normalizedDifference(['B3', 'B5'])

Image(url=ndwi.getThumbUrl({'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF'],'dimensions': 512}))

In this example, note that the min and max parameters indicate the range of pixel values to which the palette should be applied. Intermediate values are linearly stretched.

## Masking

You can use image.updateMask() to set the opacity of individual pixels based on where pixels in a mask image are non-zero. Pixels equal to zero in the mask are excluded from computations and the opacity is set to 0 for display. The following example uses an NDWI threshold to update the mask on the NDWI layer created previously:

In [10]:
# Mask the non-watery parts of the image, where NDWI < 0.4
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
Image(url=ndwiMasked.getThumbUrl({'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF'],'dimensions': 512}))

## Mosaicking

You can use masking and `imageCollection.mosaic()` to achieve various cartographic effects. The `mosaic()` method renders layers in the output image according to their order in the input collection. The following example uses `mosaic()` to combine the masked NDWI and the false color composite and obtain a new visualization:

In [11]:
# Mosaic the layers and display 
# Load the least cloudy image of 2015 image in Acra (Ghana)
next = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(ee.Geometry.Point(0.9, 6)).filterDate('2019-01-01', '2019-01-31').sort('CLOUDY_PIXEL_PERCENTAGE').first()    

mosaic = ee.ImageCollection([first, next]).mosaic()

In [12]:
import folium
#!pip install geehydro # Life saver for plotting GEE stuff with Python!
import geehydro


# Use folium to visualize the imagery.
map = folium.Map(location=[6,0.9],zoom_start=9)
#map.setOptions('HYBRID') # To see GE map underneath

map.addLayer(mosaic, {'min': 0, 'max': 2000, 'bands':['B4','B3','B2']}, 'mosaic')

map

## Clipping

The `image.clip()` method is useful for achieving cartographic effects. The following example clips the mosaic built in the previous section to an arbitrary buffer zone:

In [13]:
# Create a circle by drawing a 20,000 meter buffer around a point.
roi = ee.Geometry.Point([0.3, 6]).buffer(20000)

# Display a clipped version of the mosaic.
map.addLayer(mosaic.clip(roi),{'min': 0, 'max': 2000, 'bands':['B4','B3','B2']},'cropped')
# Adding layer control to be able to activate/deactivate layers
folium.LayerControl().add_to(map)
# Show me
map

## Image information and metadata

To explore image bands and properties in the Code Editor, print() the image and inspect the output.

In [14]:
#Get information about the bands as a list.
bandNames = image.bandNames()
print('Band names: '+str(bandNames.getInfo())) # ee.List of band names

#Get projection information from band 1.
b1proj = image.select('B1').projection()
print('Band 1 projection: '+str(b1proj.getInfo())) # ee.Projection object

#Get scale (in meters) information from band 1.
b1scale = image.select('B1').projection().nominalScale()
print('Band 1 scale: '+str(b1scale.getInfo())) # ee.Number

#Note that different bands can have different projections and scale.
b8scale = image.select('B8').projection().nominalScale()
print('Band 8 scale: '+str(b8scale.getInfo())) # ee.Number

#Get a list of all metadata properties.
properties = image.propertyNames()
print('Metadata properties: '+str(properties.getInfo())) # ee.List of metadata properties

#Get a specific metadata property.
cloudiness = image.get('CLOUD_COVER')
print('CLOUD_COVER: '+str(cloudiness.getInfo())) # ee.Number

#Get the timestamp and convert it to a date.
date = ee.Date(image.get('system:time_start'))
print('Timestamp: '+str(date.getInfo())) # ee.Date

Band names: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'QA_PIXEL', 'QA_RADSAT', 'SAA', 'SZA', 'VAA', 'VZA']
Band 1 projection: {'type': 'Projection', 'crs': 'EPSG:32610', 'transform': [30, 0, 460785, 0, -30, 4264215]}
Band 1 scale: 30
Band 8 scale: 15
Metadata properties: ['system:version', 'system:id', 'RADIANCE_MULT_BAND_5', 'RADIANCE_MULT_BAND_6', 'RADIANCE_MULT_BAND_3', 'RADIANCE_MULT_BAND_4', 'RADIANCE_MULT_BAND_1', 'RADIANCE_MULT_BAND_2', 'WRS_TYPE', 'K2_CONSTANT_BAND_11', 'K2_CONSTANT_BAND_10', 'system:footprint', 'REFLECTIVE_SAMPLES', 'SUN_AZIMUTH', 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION', 'DATE_ACQUIRED', 'ELLIPSOID', 'STATION_ID', 'RESAMPLING_OPTION', 'ORIENTATION', 'WRS_ROW', 'RADIANCE_MULT_BAND_9', 'TARGET_WRS_ROW', 'RADIANCE_MULT_BAND_7', 'RADIANCE_MULT_BAND_8', 'IMAGE_QUALITY_TIRS', 'TRUNCATION_OLI', 'CLOUD_COVER', 'GEOMETRIC_RMSE_VERIFY', 'COLLECTION_CATEGORY', 'GRID_CELL_SIZE_REFLECTIVE', 'CLOUD_COVER_LAND', 'GEOMETRIC_RMSE_MODEL', 'COLLECTIO

## Mathematical operators

Earth Engine supports many basic mathematical operators. They share some common features. Earth Engine performs math operations per pixel. When an operator is applied to an image, it's applied to each unmasked pixel of each band. In the case of operations on two images, the operation is only applied at the locations where pixels in both images are unmasked. Earth Engine automatically matches bands between images. When an operator is applied to two images, the images are expected to have the same number of bands so they can be matched pairwise. However, if one of the images has only a single band, it is matched with all of the bands in the other image, essentially replicating that band enough times to match the other image.

For a simple example, consider the task of creating the Normalized Difference Vegetation Index (NDVI) using Landsat imagery:

In [15]:
#!pip install pprint
import pprint

# Configure the pretty printing output & initialize earthengine.
pp = pprint.PrettyPrinter(depth=4)

# Load two 5-year Landsat 7 composites.
landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Compute NDVI the hard way.
ndvi1999 = (landsat1999.select('B4').subtract(landsat1999.select('B3'))
    .divide(landsat1999.select('B4').add(landsat1999.select('B3'))).rename(['NDVI']))

# Compute NDVI the easy way.
ndvi2008 = landsat2008.normalizedDifference(['B4', 'B3']).rename(['NDVI'])

print('NDVI 1999')
pp.pprint(ndvi1999.getInfo())

print('\n'+'NDVI 2008')
pp.pprint(ndvi2008.getInfo())

NDVI 1999
{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [0.00026949456294719256,
                              0,
                              -180,
                              0,
                              -0.00026949456294719256,
                              86.0000269494563],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'id': 'NDVI'}],
 'type': 'Image'}

NDVI 2008
{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [0.00026949456294719256,
                              0,
                              -180,
                              0,
                              -0.00026949456294719256,
                              86.0000269494563],
            'data_type': {'max': 1,
                          'min': -1,
                          'precision': 'float',
                          'type': 'PixelType'},
            'dimensions': [1335834, 638232],
            'id': 'NDVI'}],
 'type': 'Image'}


In [16]:
ndvi_diff = ndvi2008.subtract(ndvi1999)
map2 = folium.Map()
map2.addLayer(ndvi_diff, {'min': -0.1, 'max': 0.1, 'palette': ['ffffcc', '800026'],}, 'ndvi_diff') # Palette code for YlOrRd_09: min=ffffcc, max= 800026

# Colour map
import branca.colormap as cm
colormap = cm.linear.YlOrRd_09.scale(-0.1, 0.1)
colormap.caption = 'NDVI Difference 2008 - 1999'
map2.add_child(colormap)
map2

Mathematical operators perform basic arithmetic operations on image bands. The normalized difference operation is so common in remote sensing, Earth Engine provides a shortcut method, as shown in the second part of the example. Subtracting the images in this example results in a "change vector" for each pixel. Bands are matched automatically to perform the difference:

In [17]:
# Load two 5-year Landsat 7 composites get only 3bands from them
landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select(['B4', 'B3', 'B2'])
landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012').select(['B4', 'B3', 'B2'])

# Compute the multi-band difference image.
diff = landsat2008.subtract(landsat1999)
print('Difference')
pp.pprint(diff.getInfo())


# Compute the squared difference in each band.
squaredDifference = diff.pow(2)
print('\n'+'Squared Difference')
pp.pprint(squaredDifference.getInfo())

Difference
{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [0.00026949456294719256,
                              0,
                              -180,
                              0,
                              -0.00026949456294719256,
                              86.0000269494563],
            'data_type': {'max': 255,
                          'min': -255,
                          'precision': 'int',
                          'type': 'PixelType'},
            'id': 'B4'},
           {'crs': 'EPSG:4326',
            'crs_transform': [0.00026949456294719256,
                              0,
                              -180,
                              0,
                              -0.00026949456294719256,
                              86.0000269494563],
            'data_type': {'max': 255,
                          'min': -255,
                          'precision': 'int',
                          'type': 'PixelType'},
            'id': 'B3'},
           {'

## Expressions

To implement more complex mathematical expressions, consider using image.`expression()`, which parses a text representation of a math operation. The following example uses `expression()` to compute the Enhanced Vegetation Index (EVI):

In [19]:
# Load a Landsat 8 image.
image5 = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20200505');

# Compute the EVI using an expression.
evi = image5.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image5.select('B5'),
      'RED': image5.select('B4'),
      'BLUE': image5.select('B2')
})

Image(url=evi.getThumbUrl({'min': -1, 'max': 1, 'palette':  ['FF0000', '00FF00'],'dimensions': 512}))

## Relational, conditional and Boolean operations

To perform per-pixel comparisons between images, use relational operators. To extract urbanized areas in an image, this example uses relational operators to threshold spectral indices, combining the thresholds with and():

In [20]:
# Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])

# Create a binary layer using logical operations.
bare = ndvi.lt(0.2) and (ndwi.lt(0))

print('Bare')
pp.pprint(bare.getInfo())

Bare
{'bands': [{'crs': 'EPSG:32610',
            'crs_transform': [30, 0, 460785, 0, -30, 4264215],
            'data_type': {'max': 1,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [7661, 7801],
            'id': 'nd'}],
 'properties': {'system:footprint': {'coordinates': [[...],
                                                     [...],
                                                     [...],
                                                     [...],
                                                     [...],
                                                     [...],
                                                     [...],
                                                     [...],
                                                     [...],
                                                     [...],
                                                     [...],
       

In [21]:
Image(url=bare.getThumbUrl({'min': 0, 'max': 1, 'palette':  ['C0C0C0', '000000'],'dimensions': 512})) # Min is silver, max is black

In [22]:
# Mask the image with itslef to obtain only the land (pixels with value==1 (or TRUE))
map3 = folium.Map(location=[37.5010,-122.1899],zoom_start=8)
newbare = bare.updateMask(bare)
map3.addLayer(newbare, {'min': 0, 'max': 0.1, 'palette': ['FF0000', '00FF00']}, 'Water vs land') 
map3

In [23]:
# Mask the image with pixels with value==0 
map4 = folium.Map(location=[37.5010,-122.1899],zoom_start=8)
opp_bare = bare.updateMask(bare.eq(0))
map4.addLayer(opp_bare, {'min': 0, 'max': 0.1, 'palette': ['FF0000', '00FF00']}, 'Water vs land') 
map4